In [ ]:
# Python packages
import sys
sys.path.append('../')
from datetime import datetime
import numpy as np
import pandas as pd
import pickle

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold, GridSearchCV, cross_validate, KFold, cross_val_score
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso, LassoCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.svm import SVR
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor

from xgboost import XGBRegressor, XGBClassifier

# Custom functions
import src.settings as settings
import src.mapper_cols as mapper_cols
from src.run_all.main_get_data import get_data
from src.run_all.main_preprocess import preprocess_data
from src.utilities.utilities import get_latest_file, list_filenames

# instellingen voor panda weergave aanpassen
pd.set_option('display.max_rows', 500) # alle rijen tonen
pd.set_option('display.max_columns', 500) # alle kolommen tonen
pd.set_option('display.width', 1000) # kolombreedte
pd.set_option("display.precision", 2)     # precisie van de kolommen aanpassen
pd.set_option('display.float_format', lambda x: '{:.15f}'.format(x)) # floats output tot 15 decimalen

In [ ]:
datapath = '../data/'
filename = 'df_get_data_WMO_WIJK_HUISHOUDENS_BEVOLKING_HEFFING_202104042111.parquet.gzip'
df_get_data_WMO = pd.read_parquet(datapath + filename)

In [ ]:
df_get_data_WMO.k40huishoudensmetlaagsteinkomen

In [ ]:
df_get_data_WMO_cop = df_get_data_WMO.reset_index().copy()

In [ ]:
df_get_data_WMO_cop[df_get_data_WMO_cop['interval']=='2019']['k40huishoudensmetlaagsteinkomen']